In [4]:
%load_ext autoreload
%autoreload 2

import sys, os
import lightning.pytorch as pl
import torch
from matplotlib import pyplot as plt
import numpy as np
import pickle

sys.path.insert(1, sys.path[0] + '/..')
from src.data.datamodule import DataModule
from src.model.setup import setup_model
from src.misc.utils import set_seed_and_precision

from src.run import parse_option, main
args = parse_option(notebook=True)
args.data_dir = '../data_dev'
args.version = 'version_1'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Train

In [5]:
args.train, args.predict = True, False
args.splits = ['train', 'val']
main(args)

Global seed set to 42
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/elias/anaconda3/envs/ribfrac/lib/python3.8/site-packages/lightning/pytorch/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(

  | Name | Type    | Params
---------------------------------
0 | net  | UNet3D  | 5.0 M 
1 | loss | BCELoss | 0     
---------------------------------
5.0 M     Trainable params
0         Non-trainable params
5.0 M     Total params
20.089    Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/elias/anaconda3/envs/ribfrac/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/Users/elias/anaconda3/envs/ribfrac/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 2: 100%|██████████| 4/4 [00:22<00:00,  5.61s/it, v_num=on_1, train_loss=0.600, train_acc=0.967, val_loss=0.669, val_acc=0.993]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 4/4 [00:22<00:00,  5.62s/it, v_num=on_1, train_loss=0.600, train_acc=0.967, val_loss=0.669, val_acc=0.993]


Predict

In [10]:
args.train, args.predict = False, True
args.splits = ['predict']
main(args)

Global seed set to 42
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '../data_dev/boxes/predict/labels/RibFrac424/patch38/box1.npy'

In [7]:
pred_dir = os.path.join(args.log_dir, args.net, args.version)

In [8]:
# load preds.pkl from pred_dir
with open(os.path.join(pred_dir, 'preds.pkl'), 'rb') as f:
    preds = pickle.load(f)

In [9]:
preds[0]

(tensor([[[[0.4870, 0.4870, 0.4870,  ..., 0.4870, 0.4870, 0.4870],
           [0.4870, 0.4870, 0.4870,  ..., 0.4870, 0.4870, 0.4870],
           [0.4870, 0.4870, 0.4870,  ..., 0.4870, 0.4870, 0.4870],
           ...,
           [0.4870, 0.4870, 0.4870,  ..., 0.4870, 0.4870, 0.4870],
           [0.4870, 0.4870, 0.4870,  ..., 0.4870, 0.4870, 0.4870],
           [0.4870, 0.4870, 0.4870,  ..., 0.4870, 0.4870, 0.4870]],
 
          [[0.4870, 0.4870, 0.4870,  ..., 0.4870, 0.4870, 0.4870],
           [0.4870, 0.4870, 0.4870,  ..., 0.4870, 0.4870, 0.4870],
           [0.4870, 0.4870, 0.4870,  ..., 0.4870, 0.4870, 0.4870],
           ...,
           [0.4870, 0.4870, 0.4870,  ..., 0.4870, 0.4870, 0.4870],
           [0.4870, 0.4870, 0.4870,  ..., 0.4870, 0.4870, 0.4870],
           [0.4870, 0.4870, 0.4870,  ..., 0.4870, 0.4870, 0.4870]],
 
          [[0.4870, 0.4870, 0.4870,  ..., 0.4870, 0.4870, 0.4870],
           [0.4870, 0.4870, 0.4870,  ..., 0.4870, 0.4870, 0.4870],
           [0.4870, 0.48

### Dev